# 変量ageクレンジング

変量ageについてクレンジングを実施します．
これは変量heightやweightの場合と殆ど同じ処理となります．
違いは小数点付数か整数かという事です．

先ず，前提として次のプログラムを実施します．

- ライブラリーの搬入
- CSVファイルを読み込んでデータフレームを生成

In [1]:
import re
import unicodedata
import pandas as pd

In [2]:
df = pd.read_csv('original_body_data.csv')
df.set_index('person',inplace=True)

*****
## データフレームの基礎情報

まずはデータフレームの行数を確認します．
そして，データフレームの一部を表示します．

```Python
df.head(5)
```

In [3]:
df.head(5)

,height,weight,age,gender,blood
person,,,,,
p001,157.7,56.8,20,Male,B
p002,169.7,58.1,53,Female,A
p003,160.6,96.8,22,Male,O
p004,'155.8',,,Female,O
p005,115.5,22.5,7,Male,NaN


*****
## 変量ageの概要

変量の概要は<font color=green>describe()</font>メソッドを発行することによって得ることができます．

```Python
df.age.describe()
```

In [4]:
df.age.describe()

count     197
unique     68
top        23
freq        7
Name: age, dtype: object

describe()メソッドの出力の最後の行に注目します．

>  
```
Name: age, dtype: object
```

これは，df.ageの各データのデータ型がobject型であることを示しています．
各要素のデータ型を調べるために，リスト内包表記とtype()関数を組み合わせます．そして，その結果をset()関数で重複を除いたデータ型の取得します．

```Python
set([type(x) for x in df.age])
```

In [5]:
set([type(x) for x in df.age])

{float, str}

df.ageの各要素はstr型とfloat型が混在していることが分かりました．
そこで，float型のデータの値を確認します．

```Python
[x for x in df.age if type(x)==float]
```

In [6]:
[x for x in df.age if type(x)==float]

[nan, nan, nan]

この出力を見ると欠損値が3ヶ所あり，そこがfloat型であることが分かります．

この先の数値化処理では，この欠損値はそのまま残していきます．

## 数値化の可能性の確認

df.ageの各要素である文字列が小数点付数として数値化可能であるかを確認する必要があります．

もし確認なしで数値への変換を<font color=green>int()</font>関数で実施したらどうなるかテストします．
まずは，df.ageの最初のデータだけを数値化してみます

```Python
int(df.age[0])
```

In [7]:
int(df.age[0])

20

このように数値化が成功しました．
そこで，リスト内包表記を使って，df.ageのデータ全ての数値化を試みます．
エラーが起こることを想定して実施するので，この処理を<font color=green>try:</font>ブロックで実施してエラーが発生した場合は例外処理でエラー内容をプリントするようにします．

```Python
try:
    [int(x) for x in df.age if type(x)==str]
except ValueError as err:
    print(err)
```

In [8]:
try:
    [int(x) for x in df.age if type(x)==str]
except ValueError as err:
    print(err)

invalid literal for int() with base 10: ' '


途中で数値化できない文字列がありエラーが発生しました．
ここで発生したエラーは，数値化できない空白文字をint()関数に渡したために起こったものです．
プログラムの処理はここで中断して例外処理になり，エラーメッセージを出力しました．

他にも数値化できないデータがあるかもしれませんので，<font color=green>isdecimal()</font>関数によって，数値化できないデータをリストアップします．

```Python
[x for x in df.age if type(x)==str and not x.isdecimal()]
```

In [9]:
[x for x in df.age if type(x)==str and not x.isdecimal()]

[' ', ' 39 ', '50 ', '\u3000３９\u3000']

この結果で，クレンジング対象となるデータが判明しました．
これらのデータを綺麗にするように処理を進めていきます．
この数値化できないデータを観察すると，次の事が課題であると思われます．

- 値が空白文字だけ
- 数字の前後に空白文字がある
- 全角で数字と空白が記載されている

これらを解消するようにクレンジングを実施していきます．

## 全角から半角への変換

データ「&yen;u3000３９&yen;u3000」について，全角を半角に変換した後にシングルクォーテーションおよび空白を除去します．

```Python
unicodedata.normalize('NFKC','\u3000３９\u3000').strip("'").strip()
```

In [10]:
unicodedata.normalize('NFKC','\u3000３９\u3000').strip("'").strip()

'39'

この操作をdf.ageの全データに適用し，その結果として数値化できないデータを表示します．

```Python
[x for x in df.age if type(x)==str and not unicodedata.normalize('NFKC',x).strip("'").strip().isdecimal() ]
```

In [11]:
[x for x in df.age if type(x)==str and not unicodedata.normalize('NFKC',x).strip("'").strip().isdecimal() ]

[' ']

これで，df.ageの値で空白文字以外は全て数値化可能になります．
df.age['p004']を例にとって，この値に対してクレンジングして数値化するプロセスを単純なコードで書くと次のようになります．
このdf.age['p004']の値は空白文字になっています．

```Python
cleansedValue = unicodedata.normalize('NFKC',df.age['p004'])
if cleansedValue.isdecimal():
    intValue = int(cleansedValue)
else:
    intValue = None
print(intValue)
```

In [12]:
cleansedValue = unicodedata.normalize('NFKC',df.age['p004'])
if cleansedValue.isdecimal():
    intValue = int(cleansedValue)
else:
    intValue = None
print(intValue)

None


これは処理を分かりやすく書いたものです．
実際は，もっとコンパクトなプログラムになります．

## データフレームでの処理

ここまでdf.ageを取り出してリスト配列としてクレンジング処理を行ってきましたが，実際にはデータフレームの更新作業となります．
よって，データフレーム内でのクレンジングを行うプログラムを作成します．
この処理は次のステップで行います．

- 全角半角変換を行いシングルクォーテーションと空白文字を除去したデータを新たにcleansedAgeという列に格納する．
- cleansedAgeの値を数値化したデータを新たにage2という列に格納する．
- 列cleansedAgeを削除する．

この処理では文字列を綺麗にした中間データを作成することによって，プログラムが複雑になることを避けています．

### 文字列の整形

まず，df.ageの値を文字列として綺麗な形に整えます．
その結果は新しい列cleansedAgeとしてデータフレームに追加します．

```Python
df['cleansedAge'] = [unicodedata.normalize('NFKC',x).strip("'").strip() if type(x)==str else x for x in df.age]
```

In [13]:
df['cleansedAge'] = [unicodedata.normalize('NFKC',x).strip("'").strip() if type(x)==str else x for x in df.age]

### 文字列の数値化

次にdf.cleansedAgeの値を数値化します．
ただし，数値化できない値については<font color=green>None</font>で置き換えます．

```Python
df['age2'] = [int(x) if type(x)==str and x.isdecimal() else None for x in df.cleansedAge]
```

In [14]:
df['age2'] = [int(x) if type(x)==str and x.isdecimal() else None for x in df.cleansedAge]

### 不要な中間データの削除

使用済みとなったdf.cleansedAge列を削除します．
列の削除には，drop()メソッドとdel命令の2通りの方法があります．
ここでは<font color=green>drop()</font>メソッドによる方法で実施します．

```Python
df.drop('cleansedAge',axis=1,inplace=True)
```

In [15]:
df.drop('cleansedAge',axis=1,inplace=True)

### クレンジング済み変量age2の確認

これで年齢データについてのクレンジングが終了したので，その結果を<font color=green>describe()</font>メソッドで確認します．

```Python
df.age2.describe()
```

In [16]:
df.age2.describe()

count    196.000000
mean      40.066327
std       16.338744
min        1.000000
25%       29.000000
50%       39.500000
75%       52.250000
max       79.000000
Name: age2, dtype: float64

この出力により，196個の全てのデータが小数点付数になっていることが分かります．
行数は200行でしたので，残りのデータは数値として値が入っていない欠損値になっています．
またデータ型がint型ではなく，float型になっています．
これは，値の中に欠損値があり，これがfloat型あるため全体がfloat型に統一されるためです．

念のためにデータフレームの先頭の数行を表示します．
新たにage2という列が追加されていることが確認できます．

```Python
df.head(4)
```

In [17]:
df.head(4)

,height,weight,age,gender,blood,age2
person,,,,,,
p001,157.7,56.8,20,Male,B,20.0
p002,169.7,58.1,53,Female,A,53.0
p003,160.6,96.8,22,Male,O,22.0
p004,'155.8',,,Female,O,NaN


これで，年齢のデータについてのクレンジングは完了です．

*****